In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import folium
from folium.plugins import HeatMap
import nltk

from nltk.sentiment import SentimentIntensityAnalyzer

In [4]:
dir = 'D:\AK_SMA\dataset'

In [5]:
files = os.listdir(dir)
files

['dutch_tweets_chunk0.json',
 'dutch_tweets_chunk1.json',
 'dutch_tweets_chunk2.json',
 'dutch_tweets_chunk3.json',
 'dutch_tweets_chunk4.json',
 'dutch_tweets_chunk5.json',
 'dutch_tweets_chunk6.json',
 'dutch_tweets_chunk7.json',
 'dutch_tweets_chunk8.json',
 'dutch_tweets_chunk9.json']

In [6]:
with open(os.path.join(dir, files[0]), 'r') as f:
    json_object = json.load(f)

df_main = pd.DataFrame(data = {}, columns=list(json_object[0].keys()))

In [7]:
df_main

,full_text,text_translation,created_at,screen_name,description,desc_translation,weekofyear,weekday,day,month,...,point,latitude,longitude,altitude,province,hisco_standard,hisco_code,industry,sentiment_pattern,subjective_pattern


In [8]:

for i in range(len(files)):
  with open(os.path.join(dir, files[i])) as f:
    json_obj = json.load(f)
    df_main = pd.concat([df_main, pd.DataFrame(json_obj)], ignore_index=True)

print(df_main.shape)

C:\Users\sunil\AppData\Local\Temp\ipykernel_21536\3973225201.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_main = pd.concat([df_main, pd.DataFrame(json_obj)], ignore_index=True)


(271342, 23)


In [10]:
df_plot = df_main[['latitude', 'longitude']]

In [11]:
df_plot=df_plot.dropna(subset=['latitude', 'longitude'])

df_plot = df_plot[(df_plot['latitude'] >= -90) & (df_plot['latitude'] <= 90)]
df_plot = df_plot[(df_plot['longitude'] >= -180) & (df_plot['longitude'] <= 180)]


In [ ]:
map_center = [df_plot['longitude'].mean(), df_plot['latitude'].mean()]
heat_map = folium.Map(location=map_center, zoom_start=4)

In [13]:

head_data = df_plot[['latitude', 'longitude']].values.tolist()
HeatMap(head_data, radius=10, blur=15, max_zoom=6).add_to(heat_map)

In [14]:

image = "dutch_heatmap.html"
heat_map.save(image)

In [15]:
df_sentiment = df_main[['text_translation']]

In [16]:
nltk.download([
"names",
"stopwords",
"state_union",
"twitter_samples",
"movie_reviews",
"averaged_perceptron_tagger",
"vader_lexicon",
"punkt",
])

[nltk_data] Downloading package names to
[nltk_data]     C:\Users\sunil\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sunil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to
[nltk_data]     C:\Users\sunil\AppData\Roaming\nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\sunil\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\sunil\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sunil\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up

True

In [19]:

sia = SentimentIntensityAnalyzer()

In [20]:
df_sentiment.dropna(inplace=True)

C:\Users\sunil\AppData\Local\Temp\ipykernel_21536\2017465376.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment.dropna(inplace=True)


In [21]:
def senti_analysis(text):
    scores = sia.polarity_scores(text)
    return scores['compound']

In [22]:
df_sentiment['sentiment_scores'] = df_sentiment['text_translation'].apply(senti_analysis)

C:\Users\sunil\AppData\Local\Temp\ipykernel_21536\2295711962.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment['sentiment_scores'] = df_sentiment['text_translation'].apply(senti_analysis)


In [23]:
def sentiments(scores):
    if scores > 0.05:
        return "positive"

    elif scores <= -0.05:
        return "negative"

    else:
        return "neutral"

In [24]:
df_sentiment['sentiment_label'] = df_sentiment['sentiment_scores'].apply(sentiments)

C:\Users\sunil\AppData\Local\Temp\ipykernel_21536\909095579.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment['sentiment_label'] = df_sentiment['sentiment_scores'].apply(sentiments)


In [26]:

df_sentiment.to_csv("sentimentAnalysis.csv", index = False)